In [1]:
import numpy as np
import pandas as pd

In [2]:
mush = pd.read_csv('data/mushrooms.csv')
mush = mush.replace('?', np.nan)
mush.dropna(axis=1, inplace=True)
target = 'class'
features = mush.columns[mush.columns!=target]
target_classes = mush[target].unique()
test = mush.sample(frac=0.3)
mush = mush.drop(test.index)
cond_prob = {}
target_class_prob = {}

In [3]:
for t in target_classes:
    mush_t = mush[mush[target]==t][features]
    target_class_prob[t] = float(len(mush_t)/len(mush))
    class_prob = {}
    
    for col in mush_t.columns:
        col_prob = {}
        
        for val, cnt in mush_t[col].value_counts().iteritems():
            pr = cnt / len(mush_t)
            col_prob[val] = pr
        class_prob[col] = col_prob
    cond_prob[t] = class_prob

In [4]:
def calc_probs(x):
    probs = {}
    for t in target_classes:
        p = target_class_prob[t]
        for col, val in x.iteritems():
            try:
                p *= cond_prob[t][col][val]
            except:
                p = 0
        probs[t] = p
    return probs
            

In [5]:
def classify(x):
    probs = calc_probs(x)
    max = 0
    max_class = ''
    for cl, pr in probs.items():
        if pr > max:
            max = pr
            max_class = cl
    return max_class

In [6]:
b = []
for i in mush.index:
    b.append(classify(mush.loc[i,features]) == mush.loc[i, target] )
print(sum(b), 'correct of', len(mush))
print('Accuracy', sum(b)/len(mush))

5674 correct of 5687
Accuracy 0.9977140847547037


In [7]:
b = []
for i in test.index:
    b.append(classify(test.loc[i,features]) == test.loc[i, target] )
print(sum(b), 'correct of', len(test))
print('Accuracy', sum(b)/len(test))

2435 correct of 2437
Accuracy 0.9991793188346327
